## **Notebook PC#06**
## Hopfield Network for the Travelling Salesperson Problem (TSP)<BR>
Code based on [this content](https://github.com/ItsWajdy/Hopfield_Network) and on [this content](https://gist.github.com/payoung/6087046). <BR>

**Professor:** Fernando J. Von Zuben <br>
**Aluno(a):** <br>
**Aluno(a):**

In [ ]:
from random import randint
from random import uniform

import numpy as np

def kronecker_delta(x, y):
    if x == y:
        return 1
    return 0

class Hopfield:
    def __init__(self, cities, d, alpha):
        self.cities = cities
        self.neurons = cities**2
        self.d = d
        self.alpha = alpha

        self.w = np.zeros([self.neurons, self.neurons])

        self.x_to_index = {}
        self.index_to_x = {}

        cnt = 0
        for i in range(cities):
            for j in range(cities):
                self.index_to_x[cnt] = (i, j)
                self.x_to_index[(i, j)] = cnt
                cnt += 1

    def train(self, A, B, C, D):
        n = self.cities
        for i in range(n):
            for a in range(n):
                for j in range(n):
                    for b in range(n):
                        wc = -C
                        wa = -A*(1-kronecker_delta(a, b))*kronecker_delta(i, j)
                        wb = -B*(1-kronecker_delta(i, j))*kronecker_delta(a, b)
                        wd = -D*self.d[i][j]*(1-kronecker_delta(i, j))*(kronecker_delta(a-1, b) + kronecker_delta(a+1, b))

                        self.w[self.x_to_index[(i, a)]][self.x_to_index[(j, b)]] = wa + wb + wc + wd

    def predict(self, A, B, C, D, max_iterations):
        self.train(A, B, C, D)

        u = np.zeros([self.neurons, 1])
        for i in range(self.cities):
            for j in range(self.cities):
                u[i][0] = uniform(0, 0.03)

        prev_error = self.calc_error(u, A, B, C, D)
        repeated = 0
        max_repeat = 15
        for iteration in range(max_iterations):
            u = self.update(u, C)
            error = self.calc_error(u, A, B, C, D)
            if error == prev_error:
                repeated += 1
            else:
                repeated = 0

            if repeated > max_repeat:
                break
            prev_error = error
        ret = np.zeros([self.cities, self.cities])
        for i in range(self.cities):
            for j in range(self.cities):
                ret[i][j] = u[self.x_to_index[(i, j)]][0]

        return ret

    def update(self, u, C):
        # update is done asynchronously
        # to make update synchronous replace C*(n+1) with a bias vector containing C*(n+sigma)
        n = self.cities
        for iteration in range(5*n**2):
            i = randint(0, n-1)
            x = randint(0, n-1)
            u[self.x_to_index[(i, x)]][0] = self.f(np.dot(u.transpose(), self.w[:, self.x_to_index[(i, x)]]) + C*(n+1))
        return u

    def f(self, x):
        return 0.5*(1+np.tanh(self.alpha*x))

    def calc_error(self, u, A, B, C, D):
        tmpA = 0
        n = self.cities
        for i in range(n):
            for a in range(n):
                for b in range(n):
                    if a != b:
                        tmpA += u[self.x_to_index[(i, a)]][0] * u[self.x_to_index[(i, b)]][0]
        tmpA *= (A/2.0)

        tmpB = 0
        for i in range(n):
            for j in range(n):
                for a in range(n):
                    if i != j:
                        tmpB += u[self.x_to_index[(i, a)]][0] * u[self.x_to_index[(j, a)]][0]
        tmpB *= (B/2.0)

        tmpC = 0
        for i in range(n):
            for a in range(n):
                tmpC += u[self.x_to_index[(i, a)]][0]
        tmpC = (tmpC - n)**2
        tmpC *= (C/2.0)

        tmpD = 0
        for i in range(n):
            for j in range(n):
                for a in range(n):
                    if 0 < a < n-1:
                        tmpD += self.d[i][j]*u[self.x_to_index[(i, a)]][0]*(u[self.x_to_index[(j, a+1)]][0] + u[self.x_to_index[(j, a-1)]][0])
                    elif a > 0:
                        tmpD += self.d[i][j]*u[self.x_to_index[(i, a)]][0]*(u[self.x_to_index[(j, a-1)]][0] + u[self.x_to_index[(j, 0)]][0])
                    elif a < n-1:
                        tmpD += self.d[i][j]*u[self.x_to_index[(i, a)]][0]*(u[self.x_to_index[(j, a+1)]][0] + u[self.x_to_index[(j, n-1)]][0])
        tmpD *= (D/2.0)

        return tmpA + tmpB + tmpC + tmpD

In [ ]:
import matplotlib.pyplot as plt

def plotTSP(paths, points, num_iters=1):

    """
    path: List of lists with the different orders in which the nodes are visited
    points: coordinates for the different nodes
    num_iters: number of paths that are in the path list

    """

    # Unpack the primary TSP path and transform it into a list of ordered
    # coordinates

    x = []; y = []
    for i in paths:
        x.append(points[i][0])
        y.append(points[i][1])

    plt.plot(x, y, 'co')

    # Set a scale for the arrow heads (there should be a reasonable default for this, WTF?)
    a_scale = float(max(x))/float(50)

    # Draw the older paths, if provided
    if num_iters > 1:

        for i in range(1, num_iters):

            # Transform the old paths into a list of coordinates
            xi = []; yi = [];
            for j in paths[i]:
                xi.append(points[j][0])
                yi.append(points[j][1])

            plt.arrow(xi[-1], yi[-1], (xi[0] - xi[-1]), (yi[0] - yi[-1]),
                    head_width = a_scale, color = 'r',
                    length_includes_head = True, ls = 'dashed',
                    width = 0.001/float(num_iters))
            for i in range(0, len(x) - 1):
                plt.arrow(xi[i], yi[i], (xi[i+1] - xi[i]), (yi[i+1] - yi[i]),
                        head_width = a_scale, color = 'r', length_includes_head = True,
                        ls = 'dashed', width = 0.001/float(num_iters))

    # Draw the primary path for the TSP problem
    plt.arrow(x[-1], y[-1], (x[0] - x[-1]), (y[0] - y[-1]), head_width = a_scale,
            color ='g', length_includes_head=True)
    for i in range(0,len(x)-1):
        plt.arrow(x[i], y[i], (x[i+1] - x[i]), (y[i+1] - y[i]), head_width = a_scale,
                color = 'g', length_includes_head = True)

    #Set axis too slitghtly larger than the set of x and y
    plt.xlim(0, max(x)*1.1)
    plt.ylim(0, max(y)*1.1)
    plt.show()

In [ ]:
import numpy as np

def calc_d(cities):
	n = cities.shape[0]
	d = np.zeros([n, n])
	for i in range(n):
		for j in range(n):
			d[i][j] = np.sqrt(np.square(cities[i][0] - cities[j][0]) + np.square(cities[i][1] - cities[j][1]))
	return d

def calc_dist_route(cities):
  n = cities.shape[0]
  d = 0
  for i in range(n-1):
    d += np.sqrt(np.square(cities[i][0] - cities[i+1][0]) + np.square(cities[i][1] - cities[i+1][1]))
  d += np.sqrt(np.square(cities[n-1][0] - cities[0][0]) + np.square(cities[n-1][1] - cities[0][1]))
  return d

def set_cities(mode,n):
	city = np.zeros([n, 2])
	if mode == 1:
		city[0] = (0.06, 0.70)
		city[1] = (0.08, 0.90)
		city[2] = (0.22, 0.67)
		city[3] = (0.30, 0.20)
		city[4] = (0.35, 0.95)
		city[5] = (0.40, 0.15)
		city[6] = (0.50, 0.75)
		city[7] = (0.62, 0.70)
		city[8] = (0.70, 0.80)
		city[9] = (0.83, 0.20)
	elif mode == 2:
		city[0] = (0.025, 0.125)
		city[1] = (0.150, 0.750)
		city[2] = (0.125, 0.225)
		city[3] = (0.325, 0.550)
		city[4] = (0.500, 0.150)
		city[5] = (0.625, 0.500)
		city[6] = (0.700, 0.375)
		city[7] = (0.875, 0.400)
		city[8] = (0.900, 0.425)
		city[9] = (0.925, 0.700)
	elif mode == 3:
		city[0] = (0.25, 0.16)
		city[1] = (0.85, 0.35)
		city[2] = (0.65, 0.24)
		city[3] = (0.70, 0.50)
		city[4] = (0.15, 0.22)
		city[5] = (0.25, 0.78)
		city[6] = (0.40, 0.45)
		city[7] = (0.90, 0.65)
		city[8] = (0.55, 0.90)
		city[9] = (0.60, 0.25)
	elif mode == 4:
		city = np.array([[0.39483265, 0.50242079],
       [0.49523524, 0.35349841],
       [0.98993043, 0.48457834],
       [0.22134867, 0.37933196],
       [0.54776891, 0.02742277],
       [0.76523794, 0.78075363],
       [0.08019347, 0.74317798],
       [0.16455042, 0.77782448],
       [0.41247276, 0.06330139],
       [0.59550737, 0.88921593]])
	else:
		city = np.random.rand(n, 2)
	return city

n = 10 # For options 1 to 4, please choose n = 10
city = set_cities(2,n)
d = calc_d(city)

mini = 1000
maxi = -1
max_it = 50 # Please, set the maximum number of iterations here
mini_def = mini
for iteration in range(max_it):
  print("Iteration:", iteration)
  hp = Hopfield(n, d, 50.0)
  v = hp.predict(A=100.0, B=100.0, C=90.0, D=110.0, max_iterations=500)
  print(v)

  dist = 0
  prev_row = -1
  route = np.zeros(n)
  ind = 0
  for col in range(v.shape[1]):
    if [max(i) for i in zip(*v)][col] < 1.0:
      dist += 1000
    for row in range(v.shape[0]):
      if v[row][col] == 1:
        if prev_row != -1:
          dist += d[prev_row][row]
          # print("From City {} To City {}".format(prev_row + 1, row + 1))
        else:
          row_ini = row
        prev_row = row
        route[ind] = row
        ind += 1
        break
  dist += d[row][row_ini]
  mini = min(mini, dist)
  if mini < mini_def:
    mini_def = mini
    iteration_def = iteration
    route_def = route
  maxi = max(maxi, dist)
  plotTSP(route.astype(int), city, 1)
  print(route.astype(int))
  plt.show()
  print("Distance:", dist, "\n")
print("\nMin: {}\nMax: {}".format(mini, maxi))

Execute a célula acima para ao menos 2 das instâncias disponíveis. Para cada execução, apresente o melhor resultado usando o par de células a seguir.

# **Resultados para o Caso de Estudo 1**

In [ ]:
plotTSP(route_def.astype(int), city, 1)
calc_dist_route(city[route_def.astype(int)])

In [ ]:
route_def.astype(int)

# **Resultados para o Caso de Estudo 2**

In [ ]:
plotTSP(route_def.astype(int), city, 1)
calc_dist_route(city[route_def.astype(int)])

In [ ]:
route_def.astype(int)